***GENERATED CODE FOR denialnondenial PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '995', 'mean': '', 'stddev': '', 'min': '0034CB0A-CC52-48B4-A9BE-EE93DAB6E8FD', 'max': 'FFCEDD36-A92E-4127-9535-EF1527EBDAD1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TransactionID', 'transformation_label': 'String Indexer'}], 'feature': 'TransactionID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '995', 'mean': '', 'stddev': '', 'min': '00096063-480F-4D65-9194-0997DE3B3A7C', 'max': 'FFE7E426-BD7E-4C1F-8659-4F40E9972B9F', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TransactionID'}, {'feature_label': 'TransactionID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TransactionID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '995', 'mean': '', 'stddev': '', 'min': '031D0AE3-55AE-464C-8A7F-5BF67804C20B', 'max': 'FD58DECE-1057-46A6-827D-43B0EB577036', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerID', 'transformation_label': 'String Indexer'}], 'feature': 'PayerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '995', 'mean': '', 'stddev': '', 'min': '0147E0B3-7731-485B-85BE-38EAA94CEB8F', 'max': 'FF35917B-58D1-495C-90B6-A8EB2A1AD197', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerID'}, {'feature_label': 'PayerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientID', 'transformation_label': 'String Indexer'}], 'feature': 'PatientID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '995', 'mean': '', 'stddev': '', 'min': '012FB7ED-19F5-41F3-BB5F-BBCA73BD47FB', 'max': 'FFA7B6DD-0D45-430C-897D-10BA30C36E00', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientID'}, {'feature_label': 'PatientID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run denialnondenialHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5f75cdf768ed9a8d9b98977f/part-00000-tid-5117609304810908599-245288d6-f32d-4c73-a3d6-ec7fc6691331-13000-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_Tidewater_Physicians', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT TOP(5000) dtd.ChargeDetailID, dtd.charged_amount, dtd.paid_amount, dtd.TransactionTypeID, dtd.TransactionID, pd.ProviderID, py.PayerID, p.PatientID, status=(SELECT CASE trc.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END) FROM ( SELECT cd.ChargeDetailID, cd.PatientID, tp.TransactionTypeID, t.TransactionID, sum(cd.Amount) as charged_amount, sum(t.Amount) as paid_amount FROM ChargeDetail as cd inner join Transactions as t on cd.ChargeDetailID = t.ChargeDetailID inner join TransactionType as tp on t.TransactionTypeID = tp.TransactionTypeID inner join TransactionReason as tr on t.TransactionID = tr.TransactionID inner join TransactionReasonCode as trc on tr.TransactionReasonCodeID = trc.TransactionReasonCodeID where cd.DateOfEntry between '1/1/2019' and '1/3/2019' and cd.Amount > 0.01 and exists ( select distinct ChargeDetailID from TransactionReasonCode as tr inner join TransactionReason as r on tr.TransactionReasonCodeID = r.TransactionReasonCodeID and tr.ReasonType = 'Denial' and cd.ChargeDetailID = r.ChargeDetailID) group by cd.ChargeDetailID, cd.PatientID, tp.TransactionTypeID, t.TransactionID ) as dtd inner join ChargeDetail as cd2 on dtd.ChargeDetailID = cd2.ChargeDetailID inner join TransactionType as tp on dtd.TransactionTypeID = tp.TransactionTypeID inner join TransactionReason as tr on dtd.TransactionID = tr.TransactionID inner join TransactionReasonCode as trc on tr.TransactionReasonCodeID = trc.TransactionReasonCodeID inner join Patient as p on p.PatientID = cd2.PatientID inner join Provider as pd on pd.ProviderID = cd2.ProviderID inner join Payer as py on py.PayerID = cd2.PayerId", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run denialnondenialHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "995", "mean": "", "stddev": "", "min": "0034CB0A-CC52-48B4-A9BE-EE93DAB6E8FD", "max": "FFCEDD36-A92E-4127-9535-EF1527EBDAD1", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "charged_amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "995", "mean": "1511.97", "stddev": "3195.65", "min": "7.0", "max": "65600.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "charged_amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "paid_amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "995", "mean": "259.11", "stddev": "1092.93", "min": "-11282.64", "max": "17728.24", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "paid_amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TransactionTypeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "995", "mean": "1.52", "stddev": "0.5", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "TransactionTypeID"}, {"transformationsData": [{"feature_label": "TransactionID", "transformation_label": "String Indexer"}], "feature": "TransactionID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "995", "mean": "", "stddev": "", "min": "00096063-480F-4D65-9194-0997DE3B3A7C", "max": "FFE7E426-BD7E-4C1F-8659-4F40E9972B9F", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TransactionID"}, {"transformationsData": [{"feature_label": "ProviderID", "transformation_label": "String Indexer"}], "feature": "ProviderID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "995", "mean": "", "stddev": "", "min": "031D0AE3-55AE-464C-8A7F-5BF67804C20B", "max": "FD58DECE-1057-46A6-827D-43B0EB577036", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderID"}, {"transformationsData": [{"feature_label": "PayerID", "transformation_label": "String Indexer"}], "feature": "PayerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "995", "mean": "", "stddev": "", "min": "0147E0B3-7731-485B-85BE-38EAA94CEB8F", "max": "FF35917B-58D1-495C-90B6-A8EB2A1AD197", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerID"}, {"transformationsData": [{"feature_label": "PatientID", "transformation_label": "String Indexer"}], "feature": "PatientID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "995", "mean": "", "stddev": "", "min": "012FB7ED-19F5-41F3-BB5F-BBCA73BD47FB", "max": "FFA7B6DD-0D45-430C-897D-10BA30C36E00", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "995", "mean": "0.41", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}, {"feature": "ChargeDetailID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "995", "mean": "301.1", "stddev": "231.4", "min": "0.0", "max": "751.0", "missing": "0"}, "updatedLabel": "ChargeDetailID_stringinde..."}, {"feature": "TransactionID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "995", "mean": "391.58", "stddev": "272.16", "min": "0.0", "max": "878.0", "missing": "0"}, "updatedLabel": "TransactionID_stringindex..."}, {"feature": "ProviderID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "995", "mean": "33.38", "stddev": "32.5", "min": "0.0", "max": "134.0", "missing": "0"}, "updatedLabel": "ProviderID_stringindexer_..."}, {"feature": "PayerID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "995", "mean": "3.32", "stddev": "6.27", "min": "0.0", "max": "41.0", "missing": "0"}, "updatedLabel": "PayerID_stringindexer_tra..."}, {"feature": "PatientID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "995", "mean": "202.96", "stddev": "183.94", "min": "0.0", "max": "600.0", "missing": "0"}, "updatedLabel": "PatientID_stringindexer_t..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run denialnondenialHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["ChargeDetailID_stringindexer", "charged_amount", "paid_amount", "TransactionTypeID", "TransactionID_stringindexer", "ProviderID_stringindexer", "PayerID_stringindexer", "PatientID_stringindexer"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

